# **FACT ORDERS**

**Data Reading**

In [0]:
df=spark.sql("select * from databricks_cata.silver.orders_silver")
df.limit(10).display()
df.count()

order_id,customer_id,product_id,order_date,quantity,total_amount,year
O09991,C01696,P0138,2024-07-10T00:00:00.000Z,1,452.81,2024
O09992,C00203,P0055,2024-11-14T00:00:00.000Z,3,2776.17,2024
O09993,C00568,P0008,2023-01-04T00:00:00.000Z,2,2502.52,2023
O09994,C01157,P0160,2023-04-10T00:00:00.000Z,2,3084.92,2023
O09995,C01824,P0036,2023-01-21T00:00:00.000Z,5,8901.85,2023
O09996,C01204,P0054,2024-07-26T00:00:00.000Z,4,5385.04,2024
O09997,C01459,P0493,2024-04-04T00:00:00.000Z,5,1478.5,2024
O09998,C00997,P0099,2023-01-03T00:00:00.000Z,1,1110.7,2023
O09999,C01108,P0185,2024-06-01T00:00:00.000Z,1,1595.17,2024
O10000,C00813,P0044,2024-09-07T00:00:00.000Z,2,1160.64,2024


10000

**Reading dimproducts and dimcustomers data**

In [0]:
df_dimcus=spark.sql("select dimcustomerkey, customer_id as dim_customer_id from databricks_cata.gold.dimcustomers")
df_dimpro=spark.sql("select product_id as dimproductkey, product_id as dim_product_id from databricks_cata.gold.dimproducts")

**Fact DataFrame**

In [0]:
df_fact=df.join(df_dimcus,df["customer_id"]==df_dimcus["dim_customer_id"],how="left").join(df_dimpro,df["product_id"]==df_dimpro["dim_product_id"],how="left")

df_fact_new=df_fact.drop("dim_customer_id","customer_id","dim_product_id","product_id")

In [0]:
df_fact_new.limit(10).display()
df_fact_new.count()

order_id,order_date,quantity,total_amount,year,dimcustomerkey,dimproductkey
O09991,2024-07-10T00:00:00.000Z,1,452.81,2024,393,P0138
O09992,2024-11-14T00:00:00.000Z,3,2776.17,2024,744,P0055
O09993,2023-01-04T00:00:00.000Z,2,2502.52,2023,147,P0008
O09994,2023-04-10T00:00:00.000Z,2,3084.92,2023,888,P0160
O09995,2023-01-21T00:00:00.000Z,5,8901.85,2023,982,P0036
O09996,2024-07-26T00:00:00.000Z,4,5385.04,2024,103,P0054
O09997,2024-04-04T00:00:00.000Z,5,1478.5,2024,1267,P0493
O09998,2023-01-03T00:00:00.000Z,1,1110.7,2023,1709,P0099
O09999,2024-06-01T00:00:00.000Z,1,1595.17,2024,176,P0185
O10000,2024-09-07T00:00:00.000Z,2,1160.64,2024,1799,P0044


10000

**Upsert on Fact Table**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_cata.gold.FactOrders"):
    dlt_obj=DeltaTable.forName(spark,"databricks_cata.gold.FactOrders")

    dlt_obj.alias("trg").merge(df_fact_new.alias("src"),"trg.order_id=src.order_id and trg.Dimcustomerkey=src.Dimcustomerkey and trg.Dimproductkey=src.Dimproductkey")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()
else:
    df_fact_new.write.format("delta")\
        .option("path","abfss://gold@databricksaniete.dfs.core.windows.net/FactOrders")\
        .saveAsTable("databricks_cata.gold.FactOrders")

In [0]:
%sql
select * from databricks_cata.gold.factorders limit 10

order_id,order_date,quantity,total_amount,year,dimcustomerkey,dimproductkey
O09991,2024-07-10T00:00:00.000Z,1,452.81,2024,393,P0138
O09992,2024-11-14T00:00:00.000Z,3,2776.17,2024,744,P0055
O09993,2023-01-04T00:00:00.000Z,2,2502.52,2023,147,P0008
O09994,2023-04-10T00:00:00.000Z,2,3084.92,2023,888,P0160
O09995,2023-01-21T00:00:00.000Z,5,8901.85,2023,982,P0036
O09996,2024-07-26T00:00:00.000Z,4,5385.04,2024,103,P0054
O09997,2024-04-04T00:00:00.000Z,5,1478.5,2024,1267,P0493
O09998,2023-01-03T00:00:00.000Z,1,1110.7,2023,1709,P0099
O09999,2024-06-01T00:00:00.000Z,1,1595.17,2024,176,P0185
O10000,2024-09-07T00:00:00.000Z,2,1160.64,2024,1799,P0044


In [0]:
%sql
SELECT count(*) FROM databricks_cata.gold.factorders

count(*)
10000
